In [5]:
import networkx as nx
import numpy as np
import pandas as pd

In [67]:
def create_network(df, year):
    
    # returns a Graph object corresponding to the international relations network of the given year
    # edges have a weighted attribute which can be 1 or -1
    
    df = df[df['year']==year]   # select one particular year
    df = df[(df['alliance'] != 0) | (df['conflict']!=0)]   #filter entries with no link
    
    df['weight'] = df['alliance']+df['conflict']+df['alliance']*df['conflict']   # merge alliances and conflicts in a single column
    df = df.drop(columns = ['alliance','conflict'])
    
    df_negative = df[df['weight'] == -1]    # df with only negative edges
         
    # create the networks
    
    G = nx.from_pandas_edgelist(df, source ='statea', target = 'stateb', edge_attr='weight')
    G_neg = nx.from_pandas_edgelist(df_negative, source ='statea', target = 'stateb')
    
    return G, G_neg



def find_largest_cliques(G, G_neg, countries_to_contain = []):
    
    ###  find the largest clique in G
    
    #find all cliques in G and Gneg
    Cliques = nx.find_cliques(G)
    
    # find biggest clique
    max_clique = []
    for clique in Cliques:
        
        if np.all(np.isin(countries_to_contain, clique)) == True:  # check that the clique contains the countries that we want
        
            if len(clique) > len(max_clique):
                max_clique = clique
    
    if G_neg is None:
        
        return max_clique
    
    else:
        # find largest negative clique contained in the largest clique
        Neg_Cliques = nx.find_cliques(G_neg)

        max_neg_clique = []
        for neg_clique in Neg_Cliques:

            if np.all(np.isin(neg_clique, max_clique)) == True:  #check whether neg clique is within the largest clique

                if len(neg_clique) > len(max_neg_clique):  # take the largest clique
                    max_neg_clique = neg_clique
    
    
        return max_clique, max_neg_clique




def find_neg_edges_outside(G, max_neg_clique):
    
    # function to find negative edges of G outside of the clique induces by the nodes in the list max_neg_clique
    
    neg_edges_outside = []   # initialize array
    for node1 in G.nodes():
        for node2 in G.nodes():
            
            if node1 != node2:
                if node1 not in max_neg_clique or node2 not in max_neg_clique:   # check edges outside the negative clique
                    if G[node1][node2]['weight']<0:   # if the edge is negative

                        neg_edges_outside.append([node1, node2])   # counting just the first node is enough since each edge appears twice

    return neg_edges_outside



def filter_negative_edges(G, max_neg_clique):
    
    # this function removes iteratively the nodes with the largest number of negative edges outside of the negative clique, until only a K_n(K_l-) structure remains 
    
    ## First filter: ##
    
    # find all negative links outside of the negative clique
    neg_edges_outside = find_neg_edges_outside(G, max_neg_clique)
    
    # if there is a negative edge between a member of the clique and a node outside the clique, remove the node outside the clique
    to_remove = set()
    for [node1, node2] in neg_edges_outside:
        if node1 in max_neg_clique:
            to_remove.add(node2)
            
    G.remove_nodes_from(to_remove)  # remove nodes from graph
    
    # update the number of negative edges after the filtering
    neg_edges_outside = find_neg_edges_outside(G, max_neg_clique)
    no_neg_edges_outside = len(neg_edges_outside)

    
    ## Second filter: ##
    #remove iteratively all nodes with negative edges outside the negative clique
    
    while no_neg_edges_outside > 0:
        
        # order nodes by number of negative elements (by converting to pandas dataframe, sorting and converting back to list)
        neg_edges_outside = [node1 for [node1, node2] in neg_edges_outside]   # take only the first element of each column
        neg_edges_df = pd.DataFrame(neg_edges_outside, columns = ['country'])
        occurences = pd.Series(neg_edges_df.groupby(by = 'country', as_index = True).size()).sort_values(ascending = False)   # count how many times a given country appears
        countries_to_remove = list(occurences.keys())

        # take the first node not in the negative clique
        for node in countries_to_remove:
            if node not in max_neg_clique:

                G.remove_node(node)
                break

        # update the number of negative edges after the filtering
        neg_edges_outside = find_neg_edges_outside(G, max_neg_clique)
        no_neg_edges_outside = len(neg_edges_outside)
      
    # return the filtered clique
    max_clique = list(G.nodes())
    
    return max_clique


In [31]:
#  FIND LARGEST POSITIVE CLIQUES  #

df = pd.read_csv('allies_and_enemies_1816_2014_iso.csv')

years_v = np.arange(1816,2015)

for q, year in enumerate(years_v):
    
    print('-----------')
    print(year)
    
    # create networks
    G, G_neg = create_network(df, year)

    # find cliques
    max_clique, max_neg_clique = find_largest_cliques(G, G_neg)
    
    # if there are no positive edges or the negative clique is smaller than a triangle, I am not interested
    if len(max_neg_clique) == len(max_clique) or len(max_clique) < 5 or len(max_neg_clique) < 3:  
        continue
        
    # filter negative edges outside the negative clique
    Gs = nx.subgraph(G, max_clique)
    max_clique = filter_negative_edges(nx.Graph(Gs), max_neg_clique)
    
            
    # print relevant information if there are positive nodes after the filtering process
    if len(max_clique) > len(max_neg_clique):
        
        # Swaziland must be changed manually because there is an incompatibility of the ISO codes
        for idx, element in enumerate(max_clique):
            if  element == 'SWA':
                max_clique[idx] = ' Swaziland'
            else: max_clique[idx] = countries.loc[element,' Country']
        
        print('Total clique after filtering:\n',[element[1:] for element in max_clique]) 
        print('No. countries:', len(max_clique))
        
        print('Negative clique:\n', max_neg_clique)



-----------
1816
-----------
1817
-----------
1818
-----------
1819
-----------
1820
-----------
1821
-----------
1822
-----------
1823
-----------
1824
-----------
1825
-----------
1826
-----------
1827
-----------
1828
-----------
1829
-----------
1830
-----------
1831
-----------
1832
-----------
1833
-----------
1834
-----------
1835
-----------
1836
-----------
1837
-----------
1838
-----------
1839
-----------
1840
-----------
1841
-----------
1842
-----------
1843
-----------
1844
-----------
1845
-----------
1846
-----------
1847
-----------
1848
-----------
1849
-----------
1850
-----------
1851
-----------
1852
-----------
1853
-----------
1854
-----------
1855
-----------
1856
-----------
1857
-----------
1858
-----------
1859
-----------
1860
-----------
1861
-----------
1862
-----------
1863
-----------
1864
-----------
1865
-----------
1866
-----------
1867
-----------
1868
-----------
1869
-----------
1870
-----------
1871
-----------
1872
-----------
1873
-----------
18

Total clique after filtering:
 ['Congo', 'Tanzania', 'Nigeria', 'Democratic Republic of the Congo', 'Zambia', 'Comoros', 'Niger', 'Mauritius', 'Ghana', 'South Africa', 'Gambia', 'Senegal', 'Togo', 'Kenya', 'Algeria', 'Equatorial Guinea', 'Angola', 'Guinea-Bissau', 'Sudan', 'Mauritania', 'Ivory Coast', 'Rwanda', 'Mozambique', 'Tunisia', 'Madagascar', 'Ethiopia', 'Eritrea', 'Burundi', 'Sierra Leone', 'Namibia', 'Cameroon', 'Guinea', 'Liberia', 'Seychelles', 'Swaziland', 'Mali', 'Cape Verde', 'Botswana', 'Sao Tome and Principe', 'Benin', 'Burkina Faso', 'Lesotho', 'Zimbabwe']
No. countries: 43
Negative clique:
 ['SDN', 'ERI', 'ETH']
-----------
2012
-----------
2013
Total clique after filtering:
 ['Congo', 'Tanzania', 'Nigeria', 'Zambia', 'Comoros', 'Niger', 'Mauritius', 'Ghana', 'South Africa', 'Gambia', 'Senegal', 'Togo', 'Kenya', 'Algeria', 'Equatorial Guinea', 'Guinea-Bissau', 'Sudan', 'Mauritania', 'Rwanda', 'Mozambique', 'Tunisia', 'Madagascar', 'Ethiopia', 'Eritrea', 'Burundi', 'Si

In [68]:
# the case of Yugoslavia

subgraph_countries = ['ARM', 'KGZ', 'HRV', 'TKM', 'CYP', 'MDA', 'MKD', 'MCO', 'LIE', 'MLT', 'AND', 'YUG', 'BIH', 'TJK', 'LVA', 'UZB', 'BLR', 'SMR']


for year in range(1992,2004):
    G, G_neg = create_network(df, year)
    G = nx.Graph(nx.subgraph(G, subgraph_countries))
    
    # I want the cliques containing YUG, HRV and BIH
    max_clique = find_largest_cliques(G, G_neg = None, countries_to_contain = ['YUG', 'HRV', 'BIH'])
    print('-----')
    print(year,':',max_clique)
    print('No countries:',len(max_clique))
    
    if G['YUG']['HRV']['weight'] == -1 and G['YUG']['BIH']['weight'] == -1 and G['BIH']['HRV']['weight'] == -1:
        
        print('Negative triangle YUG-HRV-BIH present')
        
    else:
        
        print('NO NEGATIVE TRIANGLE')

-----
1992 : ['ARM', 'TJK', 'SMR', 'HRV', 'MLT', 'LVA', 'LIE', 'TKM', 'YUG', 'CYP', 'BIH']
No countries: 11
Negative triangle YUG-HRV-BIH present
-----
1993 : ['YUG', 'HRV', 'BIH']
No countries: 3
Negative triangle YUG-HRV-BIH present
-----
1994 : ['HRV', 'BIH', 'YUG']
No countries: 3
Negative triangle YUG-HRV-BIH present
-----
1995 : ['HRV', 'BIH', 'YUG']
No countries: 3
Negative triangle YUG-HRV-BIH present
-----
1996 : ['HRV', 'BLR', 'MKD', 'BIH', 'YUG']
No countries: 5
Negative triangle YUG-HRV-BIH present
-----
1997 : ['HRV', 'BLR', 'MKD', 'BIH', 'YUG']
No countries: 5
Negative triangle YUG-HRV-BIH present
-----
1998 : ['HRV', 'BLR', 'MKD', 'BIH', 'YUG']
No countries: 5
Negative triangle YUG-HRV-BIH present
-----
1999 : ['HRV', 'BLR', 'MKD', 'BIH', 'YUG']
No countries: 5
Negative triangle YUG-HRV-BIH present
-----
2000 : ['ARM', 'MCO', 'KGZ', 'HRV', 'SMR', 'MLT', 'TJK', 'LIE', 'LVA', 'UZB', 'TKM', 'AND', 'CYP', 'BLR', 'MKD', 'YUG', 'BIH', 'MDA']
No countries: 18
Negative triangle 